In [ ]:
# Требуется написать SVM без библиотек
# посчитать у него f-меру и confusion матрицу. 
# Разобраться в том, что такое критерий Вилкоксона, и сравнить с его помощью kNN и SVM, посчитать p-value. 
# Всё это на старом наборе данных chips.txt из первой лабы.  

# Уметь отвечать на любую теорию, как про SVM, так и про статистический тест.

In [ ]:
%autosave 15
%matplotlib inline

import numpy as np
import scipy as sp
import pandas as pd
import math
import random

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [20]:
def getData():
    fin = open('chips.txt', 'r')
    lines = fin.readlines()
    fin.close()
    
    classes = dict()
    data = list()
    
    for x, y, z in [map(float, x.split(',')) for x in lines]:
        data.append((x, y))
        classes[(x, y)] = z
    
    return (data, classes)

originData, trueClasses = getData()

118


In [ ]:
def printAllPoints(data, classes):
    green = [[], []]
    blue = [[], []]
    for (x, y) in data:
        if (classes[(x, y)] == 0):
            green[0].append(x)
            green[1].append(y)
        else:
            blue[0].append(x)
            blue[1].append(y)
    plt.plot(green[0], green[1], 'g.', blue[0], blue[1], 'b.')

In [ ]:
def getConfusion(pClasses):
    true_positive = [0, 0]
    false_positive = [0, 0]
    
    for point in originData:
        trueClass = trueClasses[point]
        predictedClass = pClasses[point]
        
        if predicted == real_class:
            true_positive[predicted] += 1
        else:
            false_positive[predicted] += 1
    
    return (true_positive, false_negative)
    
    

def getF1Score(true_positive, false_positive):
    all_points = [0, 0]
    all_points[0] = true_positive[0] + false_positive[1]
    all_points[1] = true_positive[1] + false_positive[0]
       
    if true_positive[1] > 0:
        recall = true_positive[1] / all_points[1]
        precision = true_positive[1] / (true_positive[1] + false_positive[1])
        # F1 measure
        return 2 * (precision * recall) / (precision + recall)
    return 0

In [ ]:
# H0: difference between predicted classes follows a symmetric distribution around zero
# H1: difference between predicted does not follow a symmetric distribution around zero.

# reference link: https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test

def getWilcoxonRank(clases1, classes2):
    diff = 0
    n = 0
    for point in originData: 
        diff += classes1[point] - classes2[point]
        if (classes1[point] - classes2[point] != 0):
            n += 1
    
    # two-sided, for one-sided should count only + or - values
    
    diff *= (n + 1) / 2  # домножаем на ранг. все значения по модулю = 1 => ранг каждого - (1 + 2 + .. + n) / n 
    
    
    #Find the "area beyond z", using a table of areas under the normal curve (such as the one on my website). 
    #The area beyond z = 0.0099. 
    #In other words, values of W as large as ours are likely to occur by chance with a probability of 0.0099. 
    #This is the one-tailed probability of obtaining W however; 
    # we were merely seeing whether there was a difference in the ratings of Ant and Dec (a two-tailed, or non-directional, hypothesis), 
    # so we need to multiply this probability by 2, to get p = .019, which rounds to p = .02.
    
    if (n > 20):
        # z_score
        mean = n * (n + 1) / 4
        sd = (n * (n + 1) * (2 * n + 1) / 24) ** 0.5
        zScore = (diff - mean) / sd
        print("ZScore: ", zScore)
    else:
        # ok
        print("WScore: ", wScore)

In [ ]:
def k_fold_cv():
    cData = originData[::]
    random.shuffle(cData)
    foldSize = len(cData) // k
    folds = [cData[i * foldSize : min((i + 1) * foldSize, len(cData))] for i in range(k)]
    return folds


# solution should be a function with two arguments: learningSet, testingSet, which returns dict classes[(x, y)]
def getResult(solution):
    predictedClasses = dict()
    
    folds = k_fold_cv()
    
    for i in range(len(folds)):
        learningSet = folds[0:i] + folds[i + 1:]
        testingSet = folds[i:i + 1] 
        predictedClasses.update(solve(learingSet, testingSet))
    
    tp, fp = getConfusion(predictedClasses)
    f1 = getF1Score((tp, fp))
    
    return predictedClasses, (tp, fp), getF1Score(predictedClasses)

In [ ]:
def toPolar(x, y, x0, y0):
    x, y = x - x0, y - y0
    r = (x ** 2 + y ** 2) ** (1 / 2)
    a = math.atan2(x, y)
    return (r, a)

def getCenter():
    fin = open('chips.txt', 'r')
    lines = fin.readlines()
    fin.close()

    x0, y0 = 0, 0
    for x, y, z in [map(float, x.split(',')) for x in lines]:
        x0 += x
        y0 += y
    x0 = x0 / len(lines)
    y0 = y0 / len(lines)
    
    return (x0, y0)

In [ ]:
simpleData = get_data(lambda x, y:(x, y), '-')
sumData = get_data(lambda x, y:(x, x + y), 'x, y -> x, x + y')
polarData = get_data(lambda x, y: toPolar(x, y, 0, 0), 'polar(0, 0)')

center = getCenter()
polarData2 = get_data(lambda x, y: toPolar(x, y, center[0], center[1]), 'polar(center_x, center_y)')



wideData = get_data(lambda x, y: (toPolar(x, y, center[0], center[1])[0] * 10, 
                           toPolar(x, y, center[0], center[1])[0] + toPolar(x, y, center[0], center[1])[1]
                          ), 
             'wide')
printAllPoints(wideData[0], wideData[1])


data = [simpleData, sumData, polarData, polarData2, wideData]

In [ ]:
printAllPoints(wideData[0], wideData[1])

In [ ]:
printAllPoints(simpleData[0], simpleData[1])

In [ ]:
printAllPoints(sumData[0], sumData[1])

In [ ]:
printAllPoints(polarData[0], polarData[1])

In [ ]:
printAllPoints(polarData2[0], polarData2[1])

In [ ]:
def minkowskiDistance(x, y, p):
    res = 0
    for i in range(len(x)):
        res += abs(x[i] - y[i]) ** p
    return res ** (1 / p)

# https://en.wikipedia.org/wiki/Cosine_similarity
def cosineSimilarity(x, y):
    res, a, b = 0, 0, 0
    for i in range(len(x)):
        res += x[i] * y[i]
        a += x[i] ** 2
        b += y[i] ** 2
    a = a ** (1 / 2)
    b = b ** (1 / 2)
    return res / a / b
    

metrics = [lambda x, y: minkowskiDistance(x, y, 1), 
           lambda x, y: minkowskiDistance(x, y, 2)
          ]
#lambda x, y: cosineSimilarity(x, y)]    

metric_names = dict()
metric_names[metrics[0]] = 'minkowski with p = 1'
metric_names[metrics[1]] = 'minkowski with p = 2'
#metric_names[metrics[2]] = 'cosine'

#for metric in metrics:
#    print(metric((1, 1), (2, 2)))

In [ ]:
def k_fold_cv(k, length):
    one_fold_length = length // k
    others = length % k
    indexies = [i for i in range(length)]
    result = list()
    for i in range(k):
        test_suit = list()
        train_suit = list()

        for j in range(one_fold_length):
            index = indexies[int(np.random.uniform(0, len(indexies))) % len(indexies)]
            test_suit.append(index)
            indexies.remove(index)
        if others > 0:
            others -= 1
            index = indexies[int(np.random.uniform(0, len(indexies))) % len(indexies)]
            test_suit.append(index)
            indexies.remove(index)
        
        for j in range(length):
            if j not in test_suit:
                train_suit.append(j)
        
        result.append((train_suit, test_suit))
    return result

In [ ]:
def predict_class(k, metric, learn_suit, classes, point, kernel):
    distances = [(metric(l_point, point), l_point) for l_point in learn_suit]
    distances.sort()
    
    d = distances[k][0]
    if (d == 0):
        d = 1
    
    s = [0, 0]
    for j in range(k):
        dist, p = distances[j]
        s[int(classes[p])] += kernel(dist / d)
        
    if (s[0] > s[1]):
        return 0
    else:
        return 1

In [ ]:
def kNN(metric, data, classes, kernel, cv_params = (1, 10)):
    
    def compute_score(k, train_suit, test_suit):
        true_positive = [0, 0]
        false_positive = [0, 0]
        all_points = [0, 0]

        for point in test_suit:
            predicted = predict_class(k, metric, train_suit, classes, point, kernel)
            real_class = int(classes[point])
            
            if predicted == real_class:
                true_positive[predicted] += 1
            else:
                false_positive[predicted] += 1
            all_points[real_class] += 1
                
        if true_positive[1] > 0:
            recall = true_positive[1] / all_points[1]
            precision = true_positive[1] / (true_positive[1] + false_positive[1])
            # F1 measure
            return 2 * (precision * recall) / (precision + recall)
        return 0


    tfold, kfold = cv_params
    max_neighbors = 0
    max_score = 0
    for neighbors in range(2, int(np.sqrt(len(data)))):
        average_score = 0
        for i in range(tfold):
            kfold_index = k_fold_cv(kfold, len(data))
            score = 0
            for train_suit, test_suit in kfold_index:
                training_suit = [data[i] for i in train_suit]
                testing_suit = [data[i] for i in test_suit]
                score += compute_score(neighbors, training_suit, testing_suit)
            average_score += score
        
        average_score /= (tfold * kfold)
        
        if average_score > max_score:
            max_neighbors = neighbors
            max_score = average_score
    
    return (max_neighbors, max_score)

In [ ]:
kernels = [
    (lambda x: 1 / 2, 'uniform'),
    (lambda x: 1 - abs(x), 'triangular'),
    (lambda x: 3 / 4 * (1 - x * x), 'parabolic'),
    (lambda x: (1 - x ** 2) ** 2 * 15 / 16, 'quartic')
]

In [ ]:
results = pd.DataFrame(columns=['k', 'metric', 'accuracy'])


best_accuracy = 0
best_kernel   = None
best_metric   = None
best_data     = None
best_neighbor = None
best_kfold = None

pd.set_option('display.height', 250)
pd.set_option('display.max_rows', 250)

for kfold in range(5, 11):
    for kernel in kernels:
        for metric in metrics:
            for input_data, classes, transform_name, transform in data:
                k, accuracy = (kNN(metric, input_data, classes, kernel[0], (1, kfold)))
                raw = pd.DataFrame([[kfold, k, metric_names[metric], transform_name, accuracy, kernel[1]]], columns=['folds', 'k', 'metric', 'transformation name', 'accuracy', 'kernel'])
                results = results.append(raw, ignore_index=True)
            
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_kernel = kernel[0]
                    best_metric = metric
                    best_data = (input_data, classes, transform)
                    best_neighbor = k
                    best_kfold = kfold
                
                    best_string = str(accuracy) + ', ' + str(k) + ', ' + kernel[1] + ', ' + metric_names[metric] + ", " + transform_name + " folds: " + str(best_kfold)

display(results)

In [ ]:
green = [[], []]
blue = [[], []]

for x in np.arange(-1, 1.5, 0.01):
    for y in np.arange(-1, 1.5, 0.01):
        predicted = predict_class(best_neighbor, best_metric, best_data[0], best_data[1], best_data[2](x, y), best_kernel)

        if (predicted == 0):
            green[0].append(x)
            green[1].append(y)
        else:
            blue[0].append(x)
            blue[1].append(y)

plt.title(best_string)
plt.plot(green[0], green[1], 'g.', blue[0], blue[1], 'b.')

In [ ]:
printAllPoints(simpleData[0], simpleData[1])